<a href="https://colab.research.google.com/github/Zwarzen/PCVK_Genap_2022/blob/main/week11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection

metode yang digunakan :
1. Template Matching
2. Edge Detection
3. Corner Detection
4. Grid Detection
5. Contour Detection


In [1]:
# Import Lib
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# akses drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Template Matching

menggunakan lib opencv:

cv.matchTemplate(), dengan param:

- image: citra input 
- templ: template yang dicari, ukuran tidak boleh lebih besar dari input
- method: metode dari template matching 

jenis metode template matching opencv: 
1. TM_SQDIFF
2. TM_SQDIFF_NORMED
3. TM_CCORR
4. TM_CCORR_NORMED
5. TM_CCOEFF
6. TM_CCOEFF_NORMED